In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ultra-wide-band-pose-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F67478%2F7734014%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240218%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240218T100913Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7cf14ea4e398e90a1a774af0fe9a6068624dadff699e33354a369e7007451ce78b02ec77f50f7e9b922e66ec21df28cd700376191478f375740211b08b238dfd81dab55549b47eac5b1bf741b6fbfbdc97b6244d6375d7e6bc576dd1431f910210e92f80a213e5c44b1ca8670f9f2d44a6da84afc425430a3c137e87ac6afcdebba36e09589187cec9cc359897eed0106da0f88e5448532f3cc6e7649138b311e8d7319df2e280990d026bafe2eb8af60e27cc53a689386eac3bb98150a3d53811b8e6c5ea1d9c1a64900c883969463c284829a9826d904eba326fa289da0ef53858a35d974c779b168eb9987b444510fa3578ee191cb31883d4e969311e17ae,signal-to-img:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4451631%2F7638551%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240218%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240218T100914Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da455fe068eb0277b9eed240928dfa4d103fee845b1d279127eae4c45f420fef99e0df4fbd35af7b26bbe48b0ba2ddc42904835f8af2771659c2cf8c54ecf0ed30630c3a3ca1ab53b4132d1edc837c5aaf23dab4efdfe65c29393bc14e454ab9efee6c68c4ca20bdf1e5ab8ccda9561fbe55f08833abd9e74aa53ca2de95023f2509bbacbc6d810808229373dbc35245dc36e5411a5da121cd479da2dfcefd9835dd440ce9db94d5add2555e8651119bf6e0474f42ac1a00a4c6614bfc1c94da89de400fc09798a7aa9f8b97f79e15f2d51a6ad382e5625aa8fd6b57127a80be4bddfa57a09db2e0c3f93a4bf1677e2b8624b18d83c41fffd413dda33ee8e10f1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# baseline

In [ ]:
from torch.utils.data import Dataset
import os
import numpy as np
from scipy import fftpack as fft
import cv2
import pandas as pd

classes = ["fall", "jump", "not_fall", "run", "run_sit", "sit_run", "walk"]

overlap = 128
n_rd_history = 256
overlap_count = 0
min_iq = 0
max_iq = 0.05
min_rf = 0
max_rf = 45


class CSVDopplerDatagen(Dataset):
    def __init__(self,
                 csv='/kaggle/input/ultra-wide-band-pose-prediction/annotations.csv',
                 src='/kaggle/input/ultra-wide-band-pose-prediction/train/train'):
        self.csv = pd.read_csv(csv).fillna(-1)
        self.src = src

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        row = self.csv.iloc[idx]
        iq = np.load(os.path.join(
            self.src,
            row['id'] + '.npy'
        ))
        if iq.shape[0] > 2560:
            iq = iq[-2560:]
        range_frequency = self.range_frequency([iq])[0]
        image = np.stack((
            iq / max_iq,
            range_frequency / max_rf,
            np.zeros_like(iq),
        ))
        return image, row['class']

    @staticmethod
    def range_frequency(datas):
        Range_frequency_frame = []
        for data in datas:
            jitter = 1e-10
            noise_threshold = -45
            dB = True
            rd_history = np.hanning(data.shape[0])[:, None] * np.array(data)
            # Range-Doppler
            rd = fft.fft(rd_history, axis=0)
            rd = fft.fftshift(rd, axes=0)
            rd = abs(rd)
            if dB:
                rd = 20 * np.log10(rd + jitter)
                rd[rd < noise_threshold] = noise_threshold
            Range_frequency_frame.append(rd)
        return np.stack(Range_frequency_frame)



if __name__ == '__main__':
    dset = CSVDopplerDatagen()
    min_rf = float('inf')
    min_iq = float('inf')
    max_rf = -float('inf')
    max_iq = -float('inf')
    lengths = {2560: 0, 7680: 0}
    for i in range(len(dset)):
        data, _ = dset[i]
        # min_iq = min(min_iq, np.abs(data[0]).min())
        # max_iq = max(max_iq, np.abs(data[0]).max())
        # min_rf = min(min_rf, np.abs(data[1]).min())
        # max_rf = max(max_rf, np.abs(data[1]).max())
        lengths[data.shape[1]] += 1
    x = dset[0]


In [ ]:
from torch import nn
from timm.models import convnext

class ViTCls(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = convnext.convnext_base(pretrained=True)
        self.cls = nn.Linear(self.encoder.num_features, 7)

    def forward(self, images):
        x = self.encoder.forward_features(images)
        x = self.encoder.forward_head(x, pre_logits=True)
        y = self.cls(x)
        return y



# signal to img (do it in colab)

In [ ]:
!pip install timm

In [ ]:
!pip install transformers
!pip install accelerate -U

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, random_split

import timm
import timm.optim
import timm.scheduler
from timm.data import ImageDataset, create_dataset, create_loader
from timm.data.transforms_factory import create_transform

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from PIL import Image

import numpy as np

from tqdm.notebook import tqdm

import glob

In [ ]:
import scipy.fftpack as fft
def Range_frequency(datas):
    Range_frequency_frame = []
    for data in datas:
        dB = True
        # Range-Doppler
        rd = fft.fft(data, axis=0)
        rd = fft.fftshift(rd, axes=0)
        rd = abs(rd)
        DBrd = 20 * np.log10(rd+1e-10)
        Range_frequency_frame.append(DBrd)
    return np.stack(Range_frequency_frame)

def range_time(IQ_data):
    n_rd_history = 256
    frame = []
    frames = []
    for iqini in IQ_data:
        if len(frame)<n_rd_history:
            frame.append(iqini)
        else:
            frames.append(np.copy(frame))
            frame.append(iqini)
            frame = frame[1::]
    return np.stack(frames)

In [ ]:
annotations = pd.read_csv("/content/annotations.csv")
annotations.shape

In [ ]:
data = []
labels = []

for index in range(annotations.shape[0]):
    file_path = '/content/train/train/{0}.npy'.format(annotations['id'][index])
    np_data = np.load(file_path)
    data.append(np_data)
    labels.append(annotations['class'][index])

len(data), len(data[0])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.1, random_state=42, stratify=labels)
len(X_train), len(X_val)

In [ ]:
output_folder = '/content/drive/MyDrive/Colab Notebooks/data/signal_train/'
os.makedirs(output_folder, exist_ok=True)

for i in tqdm(range(len(X_train))):
    train = X_train[i]
    train = range_time(train)
    train = Range_frequency(train)
    train = train.reshape(train.shape[0] * train.shape[1], train.shape[2])
    label = y_train[i]
    label_folder = os.path.join(output_folder, str(label))
    os.makedirs(label_folder, exist_ok=True)
    file_path = os.path.join(label_folder, str(i) + '.png')
    fig = plt.figure(frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(np.abs(train.T), aspect="auto")
    fig.savefig(file_path)
    fig.clear()
    plt.close(fig)

In [ ]:
output_folder = '/content/drive/MyDrive/Colab Notebooks/data/signal_val/'
os.makedirs(output_folder, exist_ok=True)

for i in tqdm(range(len(X_val))):
    train = X_val[i]
    train = range_time(train)
    train = Range_frequency(train)
    train = train.reshape(train.shape[0] * train.shape[1], train.shape[2])
    label = y_val[i]
    label_folder = os.path.join(output_folder, str(label))
    os.makedirs(label_folder, exist_ok=True)
    file_path = os.path.join(label_folder, str(i) + '.png')
    fig = plt.figure(frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(np.abs(train.T), aspect="auto")
    fig.savefig(file_path)
    fig.clear()
    plt.close(fig)

In [ ]:
data_test = []
names = []

for index in range(sam_submit.shape[0]):
    file_path = '/kaggle/input/ultra-wide-band-pose-prediction/test/test/{0}.npy'.format(sam_submit['id'][index])
    np_data = np.load(file_path)
    data_test.append(np_data)
    names.append(sam_submit['id'][index])

len(data_test)

In [ ]:
output_folder = '/content/drive/MyDrive/Colab Notebooks/data/signal_test/'
os.makedirs(output_folder, exist_ok=True)
from tqdm import tqdm

for i in tqdm(range(len(data_test))):
    train = data_test[i]
    train = range_time(train)
    train = Range_frequency(train)
    train = train.reshape(train.shape[0] * train.shape[1], train.shape[2])
    file_path = os.path.join(output_folder, names[i] + '.png')
    fig = plt.figure(frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(np.abs(train.T), aspect="auto")
    fig.savefig(file_path)
    fig.clear()
    plt.close(fig)

# LeVIT

## load data

In [ ]:
import os
import cv2
from torchvision import transforms

X_train = []
y_train = []
size = (224, 224)
directory_path = '/kaggle/input/signal-to-img/data/signal_train'
folder_names = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]

def importdatasets_name(name):
    input_dir = "/kaggle/input/signal-to-img/data/signal_train/" + name

    input_img_paths_png = sorted(
        [
            os.path.join(input_dir, fname)
            for fname in os.listdir(input_dir)
            if fname.endswith(".png")
        ]
    )

    for i in input_img_paths_png:
      try:
        image = cv2.imread(i)
        image = cv2.resize(image, size)

        X_train.extend([image])
        y_train.extend([name])
      except:
        pass

for i in tqdm(folder_names):
  importdatasets_name(i)

In [ ]:
class_names, y_train = np.unique(np.array(y_train), return_inverse=True)
_, y_val = np.unique(np.array(y_val), return_inverse=True)
class_names

In [ ]:
import os
import cv2
from torchvision import transforms

X_val = []
y_val = []
size = (224, 224)
directory_path = '/kaggle/input/signal-to-img/data/signal_val'
folder_names = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]

def importdatasets_name(name):
    input_dir = "/kaggle/input/signal-to-img/data/signal_val/" + name

    input_img_paths_png = sorted(
        [
            os.path.join(input_dir, fname)
            for fname in os.listdir(input_dir)
            if fname.endswith(".png")
        ]
    )

    for i in input_img_paths_png:
      try:
        image = cv2.imread(i)
        image = cv2.resize(image, size)

        X_val.extend([image])
        y_val.extend([name])
      except:
        pass

for i in tqdm(folder_names):
  importdatasets_name(i)

In [ ]:
X_train = np.stack(X_train)
X_val = np.stack(X_val)
X_train.shape, X_val.shape

In [ ]:
plt.imshow(X_train[3], aspect="auto")

## model & train

In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, images, labels=None):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx]).permute(2, 0, 1).float()
        if self.labels is not None:
            label = torch.tensor(self.labels[idx])
            return {"pixel_values": image, "label": label}
        else:
            return {"pixel_values": image}

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

In [ ]:
import timm
import torch.nn as nn
from transformers import TrainingArguments, Trainer

class Vit(nn.Module):
    def __init__(self, pretrained=True, num_classes=7):
        super(Vit, self).__init__()
        self.model = timm.create_model("levit_384.fb_dist_in1k", pretrained=pretrained, num_classes=num_classes)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, pixel_values, labels=None):
        outputs = self.model(pixel_values)
        logits = outputs
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return {"logits": logits, "loss": loss}

model = Vit()

In [ ]:
from transformers import default_data_collator

training_args = TrainingArguments(
    output_dir="./levit_training_output",
    num_train_epochs=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    eval_steps=500,
    evaluation_strategy="epoch",
    weight_decay=0.01,
    logging_dir="./logs",
    learning_rate=1e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=default_data_collator,
    tokenizer=None,
)

trainer.train()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

results = trainer.predict(val_dataset)
predicted_labels = torch.argmax(torch.tensor(results.predictions), axis=1).numpy()
accuracy = accuracy_score(y_val, predicted_labels)
f1 = f1_score(y_val, predicted_labels, average="weighted")

print(f"Validation Accuracy: {accuracy}")
print(f"Validation F1 Score: {f1}")

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix(y_val, predicted_labels), annot=True, cmap='RdPu', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## test

In [ ]:
input_dir = "/kaggle/input/signal-to-img/data/signal_test"

input_img_paths_png = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".png")
    ]
)

print("Number of samples:", len(input_img_paths_png))

In [ ]:
X_test = []

for i in tqdm(input_img_paths_png):
  image = cv2.imread(i)
  image = cv2.resize(image, (224, 224))
  X_test.append(image)

In [ ]:
X_test = np.stack(X_test)
X_test.shape

In [ ]:
plt.imshow(X_test[3], aspect="auto")

In [ ]:
class TestDataset(Dataset):
    def __init__(self, images, labels=None):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx]).permute(2, 0, 1).float()
        return {"pixel_values": torch.tensor(image)}

test_dataset = TestDataset(X_test)
model.to("cpu")
model.eval()

predicted_labels = []
with torch.no_grad():
    for sample in tqdm(test_dataset):
        pixel_values = sample["pixel_values"].unsqueeze(0)
        logits = model(pixel_values)["logits"]
        predictions = torch.argmax(logits, dim=1).item()
        predicted_labels.append(predictions)
predicted_labels = np.array(predicted_labels)

In [ ]:
predicted_labels

In [ ]:
ans = class_names[predicted_labels]
print(len(ans[ans == '0']))
print(len(ans[ans == '1']))
print(len(ans[ans == '2']))
print(len(ans[ans == '3']))
print(len(ans[ans == '4']))
print(len(ans[ans == '5']))
print(len(ans[ans == '6']))

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/kaggle/input/ultra-wide-band-pose-prediction/sample_submission.csv")
df

In [ ]:
for i in range(len(df)):
    file_name = df['id'][i]
    for j in range(len(df)):
        img_path = input_img_paths_png[j].replace("/kaggle/input/signal-to-img/data/signal_test/", "").replace(".png", "")
        if file_name == img_path:
            df.loc[i, 'class'] = ans[j]
            break
df

In [ ]:
df.to_csv('levit384_100ep.csv', index=False)

# resnet

In [ ]:
import glob
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, random_split

train_files = glob.glob("/kaggle/input/signal-to-img/data/signal_train/**/*.png")
val_files = glob.glob("/kaggle/input/signal-to-img/data/signal_val/**/*.png")
test_files = glob.glob("/kaggle/input/signal-to-img/data/signal_test/*.png")

transforms = {
    "train": T.Compose([
        T.Resize((224, 224), interpolation=T.InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=torch.tensor([0.4850, 0.4560, 0.4060]), std=torch.tensor([0.2290, 0.2240, 0.2250]))
    ]),
    "test": T.Compose([
        T.Resize((224, 224), interpolation=T.InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=torch.tensor([0.4850, 0.4560, 0.4060]), std=torch.tensor([0.2290, 0.2240, 0.2250]))
    ])
}

In [ ]:
import timm
import timm.optim
import timm.scheduler
from timm.data import ImageDataset, create_dataset, create_loader
from timm.data.transforms_factory import create_transform

train_dataset = ImageDataset("/kaggle/input/signal-to-img/data/signal_train", transform=transforms["train"])
val_dataset = ImageDataset("/kaggle/input/signal-to-img/data/signal_val", transform=transforms["train"])
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model("seresnextaa101d_32x8d.sw_in12k_ft_in1k", pretrained=True, num_classes=7).to(device)

In [ ]:
num_epochs = 20
criterion = nn.CrossEntropyLoss()
optimizer = timm.optim.create_optimizer_v2(model, opt="AdamW", lr=1e-3)
optimizer = timm.optim.Lookahead(optimizer, alpha=0.5, k=5)
scheduler = timm.scheduler.create_scheduler_v2(optimizer, num_epochs=num_epochs)[0]

In [ ]:
info = {
    "metric_train": [],
    "metric_val": [],
    "train_loss": [],
    "val_loss": [],
    "best_metric_val": -999,
}

In [ ]:
from sklearn.metrics import f1_score

for epoch in range(num_epochs):
    train_loss_epoch = []
    val_loss_epoch = []

    train_preds = []
    train_targets = []

    val_preds = []
    val_targets = []

    num_updates = epoch * len(train_dataloader)

    # Train Loop
    model.train()
    for batch in tqdm(train_dataloader):
        inputs, targets = batch
        outputs = model(inputs.to(device))
        loss = criterion(outputs, targets.to(device))

        loss.backward()

        optimizer.step()
        scheduler.step_update(num_updates=num_updates)
        optimizer.zero_grad()

        train_loss_epoch.append(loss.item())
        train_preds += outputs.argmax(-1).detach().cpu().tolist()
        train_targets += targets.tolist()

    optimizer.sync_lookahead()
    scheduler.step(epoch + 1)

    # Eval Loop
    model.eval()
    with torch.no_grad():
        for batch in tqdm(val_dataloader):
            inputs, targets = batch
            outputs = model(inputs.to(device))
            loss = criterion(outputs, targets.to(device))

            # Log Values
            val_loss_epoch.append(loss.item())
            val_preds += outputs.argmax(-1).detach().cpu().tolist()
            val_targets += targets.tolist()

    # Log Data
    metric_train = f1_score(train_targets, train_preds, average="macro")
    metric_val = f1_score(val_targets, val_preds, average="macro")

    info["metric_train"].append(metric_train)
    info["metric_val"].append(metric_val)

    info["train_loss"].append(np.average(train_loss_epoch))
    info["val_loss"].append(np.average(val_loss_epoch))

    if metric_val > info["best_metric_val"] and np.average(val_loss_epoch) <= min(info["val_loss"]):
        print("New Best Score have been save!")
        info["best_metric_val"] = metric_val
        torch.save(model, "checkpoint.pt")

    print(info)
    print(f"Epoch: {epoch} | Metric: {metric_val} | Training Loss: {np.average(train_loss_epoch)} | Validation Loss: {np.average(val_loss_epoch)}")

In [ ]:
loaded_model = torch.load("/kaggle/working/checkpoint.pt")
predictions = []
references = []

loaded_model.eval()
with torch.no_grad():
    for batch in tqdm(val_dataloader):
        inputs, targets = batch
        outputs = loaded_model(inputs.to(device))

        # Log Values
        predictions += outputs.argmax(-1).detach().cpu().tolist()
        references += targets.tolist()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(references, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# results = trainer.predict(val_dataset)
# predicted_labels = torch.argmax(torch.tensor(results.predictions), axis=1).numpy()
accuracy = accuracy_score(references, predictions)
f1 = f1_score(references, predictions, average="weighted")

print(f"Validation Accuracy: {accuracy}")
print(f"Validation F1 Score: {f1}")

In [ ]:
from PIL import Image

answers_final = dict()

with torch.no_grad():
    for f in tqdm(test_files):
        key = f.split("/")[-1].split(".")[0]

        img = Image.open(f).convert("RGB")
        transformed = transforms["test"](img).unsqueeze(0).to(device)
        answers_final[key] = loaded_model(transformed).argmax(-1).item()

answers_final

In [ ]:
import pandas as pd
df = pd.read_csv("/kaggle/input/ultra-wide-band-pose-prediction/sample_submission.csv")
df

In [ ]:
for i in range(len(df)):
    file_name = df['id'][i]
    df.loc[i, 'class'] = str(answers_final[file_name])
df

In [ ]:
old = pd.read_csv("/kaggle/working/seresnext.csv")

In [ ]:
compare = pd.concat([old['id'], old['class'], df['class']], axis=1)
compare.columns=['id','old','new']
compare['old'] = compare['old'].astype('string')
compare[compare['old'] != compare['new']]


In [ ]:
print(type(compare['old'][0]))
print(type(compare['new'][0]))

In [ ]:
df.to_csv('seresnext.csv', index=False)